In [ ]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'redpanda-1:9092'
server = 'localhost:9092'


producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

# PRODUCE DATA

In [ ]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'redpanda-1:9092'
server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

In [ ]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {"number":i}
    producer.send(topic_name, value=message)
    print(f'Sent: {message}')
    time.sleep(0.05)
print(time.time()-t0)
producer.flush()
t1 = time.time()
print(f"Tool {(t1-t0)} senconds")

In [ ]:
0.6194324493408203 - 0.6194324493408203 

In [ ]:
import pandas as pd 

In [ ]:
df_green = pd.read_csv('green_tripdata_2019-10.csv')

In [ ]:
df_green_process = df_green[['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'tip_amount']]

In [ ]:
df_green_process

In [ ]:
topic_name = 'green-trips-3'
t0 = time.time()
for row in df_green_process.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic = topic_name, value=row_dict)
    time.sleep(0.5)
print(time.time()- t0)

# CONSUME DATA PYSPARK

In [ ]:
import pyspark 
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"
# SparkSession.setLevel('INFO')
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Green Trips Consumer") \
    .config('spark.jars.packages', kafka_jar_package) \
    .getOrCreate()

In [ ]:
spark

In [ ]:
# sc = pyspark.SparkContext.getOrCreate()
# log4j = sc._jvm.org.apache.log4j
# log4j.LogManager.getRootLogger().setLevel(log4j.Level.DEBUG)

In [ ]:
topic_name = 'green-trips'
server = 'redpanda-1:9092'
server = 'localhost:9092'
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", server) \
    .option("subscribe", topic_name) \
    .option('startingOffsets', 'earliest') \
    .load()

In [ ]:
green_stream.printSchema()

In [ ]:
def  peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek)

In [ ]:
query.start()

In [ ]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [ ]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")